<a id="toc"></a>
# Indice

- [Contesto e obiettivo](#contesto)
- [Setup e dipendenze](#setup)
- [Serie toy quartoraria](#toy)
- [Primo grafico interattivo](#plotly)
- [Prophet: modello base](#prophet-base)
- [Forecast interattivo](#forecast-plot)
- [Componenti estratte](#components)
- [Regressori](#regressors)
- [Micro-esercizi](#exercises)
- [Riferimenti](#refs)


<a id="contesto"></a>
## Contesto e obiettivo

Nel forecasting univariato l’idea è semplice: **usare la serie storica per prevedere come proseguirà nel futuro**. Il valore pratico arriva quando si deve prendere decisioni *prima* che i dati futuri esistano.

Nei contesti utility questo si vede per esempio su : load, consumi, portate, sensori di impianto. Lo stesso schema vale in molti altri domini: domanda di un servizio, traffico, richieste su un sistema, telemetrie.

Qui non ci interessa ancora “quanto è bravo” il modello (metriche, validazione): ci concentriamo sul flusso minimo per ottenere una previsione sensata e leggerla con gli strumenti giusti.


<a id="setup"></a>
## Setup e dipendenze

Useremo esclusivamente **Prophet** (libreria Python) e Plotly per i grafici interattivi.

- Prophet lavora con un `DataFrame` con due colonne obbligatorie:
  - `ds`: timestamp (data o data+ora)
  - `y`: valore numerico osservato

<div style="background-color:#f5f5f5; padding:10px; border-left:4px solid #999;">

**Nota importante**

Prophet non installa Plotly automaticamente. Se vuoi usare i grafici interattivi di Prophet (`plot_plotly`, `plot_components_plotly`) assicurati di avere anche `plotly` e `ipywidgets` nell’ambiente.

</div>

In [ ]:
# Se stai lavorando in un ambiente pulito (es. Colab), potresti dover installare le dipendenze.
# In un corso spesso l'ambiente è già predisposto.
#
# !pip install prophet plotly ipywidgets

import numpy as np
import pandas as pd

import plotly.express as px

# Prophet
try:
    from prophet import Prophet
    from prophet.plot import plot_plotly, plot_components_plotly
except ModuleNotFoundError as e:
    raise ModuleNotFoundError(
        "Prophet non è installato in questo ambiente. "
        "Installa con: pip install prophet (e, per i plot interattivi: plotly ipywidgets)."
    ) from e

np.random.seed(7)


<a id="toy"></a>
## Serie toy quartoraria

Costruiamo una serie artificiale ma *realistica* per vedere bene cosa fa Prophet. L’obiettivo non è simulare un impianto vero, ma avere segnali riconoscibili:

- **trend** (crescita lenta nel tempo)
- **stagionalità giornaliera** (pattern che si ripete ogni 24 ore)
- **stagionalità settimanale** (weekend diversi dai feriali)
- **cicli più lenti** (variazioni su qualche giorno)
- **rumore** (variabilità non spiegata)

Aggiungiamo anche una variabile esterna `temperature`: è un esempio classico per le utility (la domanda elettrica dipende molto dalla temperatura). La useremo più avanti come regressore.


In [ ]:
# 30 giorni di dati ogni 15 minuti
freq = "15min"
points_per_day = 24 * 60 // 15  # 96
n_days = 30

rng = pd.date_range("2025-01-01", periods=points_per_day * n_days, freq=freq)

n = len(rng)
t = np.arange(n)

# Componenti
trend = 0.01 * t  # crescita lenta

# 24 ore = 96 punti
phase_day = 2 * np.pi * (t / points_per_day)
daily = 8 * np.sin(phase_day) + 2 * np.sin(2 * phase_day)

# 7 giorni
phase_week = 2 * np.pi * (t / (points_per_day * 7))
weekly = 3 * np.sin(phase_week)

# ciclo più lento (4 giorni)
cycle = 1.5 * np.sin(2 * np.pi * (t / (points_per_day * 4)))

# temperatura (giornaliera + un po' di rumore)
temperature = 15 + 7 * np.sin(phase_day - np.pi / 3) + 0.8 * np.random.randn(n)

# weekend flag (0/1)
is_weekend = (pd.Series(rng).dt.dayofweek >= 5).astype(int).to_numpy()
weekend_effect = -3 * is_weekend

noise = 1.5 * np.random.randn(n)

# Serie osservata
baseline = 50

y = (
    baseline
    + trend
    + daily
    + weekly
    + cycle
    + 0.6 * temperature
    + weekend_effect
    + noise
)

df = pd.DataFrame(
    {
        "ds": rng,
        "y": y,
        "temperature": temperature,
        "is_weekend": is_weekend,
    }
)

df.head()


<a id="plotly"></a>
## Primo grafico interattivo

Il punto qui è solo prendere confidenza con la visualizzazione interattiva: zoom, pan, hover.

Useriamo `plotly.express` perché è la via più diretta: passi un `DataFrame`, specifichi `x` e `y`, e ottieni un grafico utilizzabile subito.


In [ ]:
fig = px.line(df, x="ds", y="y", title="Serie toy quartoraria")
fig.show()


<a id="prophet-base"></a>
## Prophet: modello base

Prophet si aspetta un `DataFrame` con colonne `ds` e `y`. Qui partiamo senza regressori: vogliamo vedere cosa riesce a ricostruire usando solo trend e stagionalità (giornaliera/settimanale).


In [ ]:
df_base = df[["ds", "y"]].copy()
df_base.head()

NOTA: è obbligatorio avere questi nomi nel dataset (metodo rename di pandas) e ds deve essere in datetime (in questo caso già lo è)

Per prima cosa inizializziamo il modello con `Prophet` e lo salviamo in m.
Qui possiamo configurare il modello per adattarlo e renderlo più performante tramite delle configurazioni:



In [ ]:
m = Prophet(
    daily_seasonality=True,
    weekly_seasonality=True,
)

# Utilizzia il modello base senza componenti aggiuntive
m = Prophet() 

`m.fit(df_base)` è il passo in cui **Prophet impara la struttura della serie storica** a partire dai dati osservati.

Il modello usa le colonne `ds` (tempo) e `y` (valore) per **scomporre automaticamente la serie** in:
- un **trend** di lungo periodo;
- **stagionalità** regolari (giornaliera, settimanale, annuale);
- eventuali **effetti dei regressori** esterni;

Durante il `fit`, Prophet costruisce internamente le feature temporali e stima i parametri del modello.  
Il risultato è un **modello addestrato**, pronto per essere usato con `predict()` per stimare il futuro o analizzare le componenti.


In [ ]:
m.fit(df_base)

Per prevedere il futuro, Prophet deve sapere **per quali istanti di tempo** fare la stima.  
Per questo si costruisce un DataFrame `future` che contiene solo le **date future** (`ds`), estese oltre i dati storici.

Con `make_future_dataframe` chiediamo a Prophet di:
- prolungare l’asse temporale già visto in fase di `fit`;
- aggiungere 2 giorni di punti a frequenza 15 minuti.

Il DataFrame `future` non contiene valori osservati.  
Quando chiamiamo `predict`, il modello usa quanto ha imparato sul passato per stimare:
- il valore atteso (`yhat`);
- un intervallo di incertezza (`yhat_lower`, `yhat_upper`).


In [ ]:

# 2 giorni di previsione (sempre a 15 minuti)
horizon_days = 2
time_points_per_day = 24 * 4  # 15 minuti

future = m.make_future_dataframe(
    periods=horizon_days * time_points_per_day,   # 2 giorni a intervalli di 15 minuti
    freq="15min"
)

In [ ]:
future.describe()

In [ ]:
future.tail()

Con `predict` chiediamo al modello di **calcolare le previsioni** per tutte le date contenute nel DataFrame `future`.

Prophet restituisce un DataFrame (`forecast`) che include, per ogni istante temporale:
- `yhat`: la previsione centrale del modello;
- `yhat_lower` e `yhat_upper`: un intervallo di incertezza attorno alla previsione.

La selezione finale mostra solo le colonne essenziali e le ultime righe, così da osservare
le stime prodotte **sulle date più future**.


In [ ]:
forecast = m.predict(future)

forecast[["ds", "yhat", "yhat_lower", "yhat_upper"]].tail()

<a id="forecast-plot"></a>
## Forecast interattivo

Nel `forecast` trovi:
- `yhat`: previsione
- `yhat_lower`, `yhat_upper`: intervallo di incertezza

Qui usiamo la funzione ufficiale di Prophet per ottenere un grafico Plotly del forecast.


In [ ]:
fig_fc = plot_plotly(m, forecast)
fig_fc.show()


<a id="components"></a>
## Componenti estratte

La vista più utile, quando inizi, è la scomposizione in componenti: trend e stagionalità.

Anche qui usiamo la funzione ufficiale di Prophet per ottenere un grafico interattivo delle componenti.


In [ ]:
fig_comp = plot_components_plotly(m, forecast)
fig_comp.show()


<a id="regressors"></a>
## Regressori

Un regressore è una variabile esterna che aiuta a spiegare la serie, ad esempio:
- temperatura (utility)
- calendario (weekend / festivi)
- driver operativi (turni, campagne, aperture)

Il vincolo pratico è semplice: **devi conoscere il regressore anche nel futuro** (oppure prevederlo con un altro modello). Prophet richiede che i regressori siano presenti sia in `fit` sia in `predict`.

<div style="background-color:#f5f5f5; padding:10px; border-left:4px solid #999;">

**Attenzione**

I regressori vanno aggiunti al modello prima del `fit`. Prophet solleva errore se un regressore è costante nello storico (non c’è nulla da stimare).

</div>

In [ ]:
df_reg = df[["ds", "y", "temperature", "is_weekend"]].copy()
df_train = df_reg.iloc[:-horizon_days * time_points_per_day]
df_test = df_reg.iloc[-horizon_days * time_points_per_day:]

m_reg = Prophet()
m_reg.add_regressor("temperature")
m_reg.add_regressor("is_weekend")

m_reg.fit(df_train)

In [ ]:
future_reg

In [ ]:
future_reg = df_reg[['ds']].copy()

# Per il futuro, costruiamo regressori "plausibili".
# In un caso reale, la temperatura arriverebbe da una previsione meteo.
#replichiamo la temperatura dei due giorni precedenti
future_reg["temperature"] = df["temperature"]
future_reg["is_weekend"] = (future_reg["ds"].dt.dayofweek >= 5).astype(int)

forecast_reg = m_reg.predict(future_reg)

fig_fc_reg = plot_plotly(m_reg, forecast_reg)
fig_fc_reg.show()


In [ ]:
fig_comp_reg = plot_components_plotly(m_reg, forecast_reg)
fig_comp_reg.show()


<a id="exercises"></a>
## Micro-esercizi

<div style="background-color:#f5f5f5; padding:10px; border-left:4px solid #999;">

**Esercizio**

Cambia l’orizzonte di previsione da **2 giorni** a **7 giorni**.

Suggerimento: modifica `horizon_days` e rigenera `future` / `future_reg`.

</div>

<div style="background-color:#f5f5f5; padding:10px; border-left:4px solid #999;">

**Esercizio**

Aumenta l’ampiezza della stagionalità giornaliera nel generatore della serie (la variabile `daily`).

Osserva come cambiano: il grafico della serie e le componenti stimate da Prophet.

</div>


<a id="refs"></a>
## Riferimenti

- Prophet — pagina principale: https://facebook.github.io/prophet/
- Quickstart (Python): https://facebook.github.io/prophet/docs/quick_start.html
- Seasonalità e regressori: https://facebook.github.io/prophet/docs/seasonality%2C_holiday_effects%2C_and_regressors.html
- Repository (code e note): https://github.com/facebook/prophet
